In [1]:
import os
os.environ["WANDB_DISABLED"] = "true"

import pandas as pd
import numpy as np
import cv2
import shutil
import yaml
import ast
import warnings
warnings.filterwarnings("ignore")

from glob import glob
from tqdm import tqdm
from IPython.display import clear_output
from sklearn.model_selection import train_test_split

In [3]:
# 데이터 불러오기
train = pd.read_csv('./structured_data/train.csv')
valid = pd.read_csv('./structured_data/valid.csv')

In [4]:
# 비엔나 코드 맨 앞에 오는 0 저장 안된 부분 처리(예시:10102->010102)
pattern = r'(.+?)_(.+?)\.jpg'
train[['Vienna_Code','etc']] = train['fileName'].str.extract(pattern)
train = train.drop(columns=['etc'])
train

,area,iscrowd,image_id,bbox,id,name,fileName,height,width,Vienna_Code,category_id,supercategory,Code_Index
0,173056,False,1003329,"[48.52, 23.89, 57.13, 118.24]",xi99e1ofj6d000ljkks697qn,하나의 별,010102_1003329.jpg,416,416,010102,1,하나의 별,0
1,173056,False,1003908,"[145.4, 193.49, 120.96, 85.57]",0xxavj7dqx3b87mflfankoh,하나의 별,010102_1003908.jpg,416,416,010102,1,하나의 별,0
2,173056,False,1003922,"[4.32, 13.58, 306.52, 302.63]",u3l5mmbenmbfmh5qoq7ex8,하나의 별,010102_1003922.jpg,416,416,010102,1,하나의 별,0
3,173056,False,1005613,"[82.7, 6.84, 255.53, 325.62]",s39tjedrg0b8hsowor97vb,하나의 별,010102_1005613.jpg,416,416,010102,1,하나의 별,0
4,173056,False,1009287,"[2.08, 17.82, 120.35, 387.95]",whqo3emkieiaj5f7pro8k,하나의 별,010102_1009287.jpg,416,416,010102,1,하나의 별,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2437338,173056,False,4120080006669,"[2.84, 122.13, 132.37, 178.47]",ae2b9ca597e640588ac934,"적색, 분홍색, 주황색",290101_4120080006669.jpg,416,416,290101,1,"적색, 분홍색, 주황색",391
2437339,173056,False,4120080008927,"[54.37, 154.54, 63.68, 75.55]",a50502ca54e343d583462c,"적색, 분홍색, 주황색",290101_4120080008927.jpg,416,416,290101,1,"적색, 분홍색, 주황색",391
2437340,173056,False,4120080008927,"[19.47, 121.81, 37.61, 37.5]",49f09d67ba2b4cf489faae,"적색, 분홍색, 주황색",290101_4120080008927.jpg,416,416,290101,1,"적색, 분홍색, 주황색",391
2437341,173056,False,4120080008929,"[72.66, 160.75, 97.63, 107.46]",099ea0dcee90486d8f1d73,"적색, 분홍색, 주황색",290101_4120080008929.jpg,416,416,290101,1,"적색, 분홍색, 주황색",391


In [5]:
# 비엔나 코드 맨 앞에 오는 0 저장 안된 부분 처리(예시:10102->010102)
pattern = r'(.+?)_(.+?)\.jpg'
valid[['Vienna_Code','etc']] = valid['fileName'].str.extract(pattern)
valid = valid.drop(columns=['etc'])
valid

,area,iscrowd,image_id,bbox,id,name,fileName,height,width,Vienna_Code,category_id,supercategory,Code_Index
0,173056,False,4120110003227,"[201.42, 166.82, 16.59, 13.07]",gancg2erhgwserimdxudur,하나의 별,010102_4120110003227.jpg,416,416,010102,1,하나의 별,0
1,173056,False,4120110003257,"[58.51, 37.92, 27.02, 79.19]",rgtbhp4duh7va0djgpsyve,하나의 별,010102_4120110003257.jpg,416,416,010102,1,하나의 별,0
2,173056,False,4120110003525,"[210.6, 169.27, 50.09, 48.87]",ruhso7jsqgrhee1hggvvs4,하나의 별,010102_4120110003525.jpg,416,416,010102,1,하나의 별,0
3,173056,False,4120110003583,"[16, 135.13, 46.12, 40.75]",sdx8c46bnjc6licnwukkf,하나의 별,010102_4120110003583.jpg,416,416,010102,1,하나의 별,0
4,173056,False,4120110003665,"[154.78, 213.86, 95.89, 92.41]",k3fymz039j3m854uiyz9fz,하나의 별,010102_4120110003665.jpg,416,416,010102,1,하나의 별,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
308126,173056,False,4120110037622,"[-0.5, 148.02, 97.41, 111.23]",f94e1f7c294e42c0915ae4,"적색, 분홍색, 주황색",290101_4120110037622.jpg,416,416,290101,1,"적색, 분홍색, 주황색",391
308127,173056,False,4120110037623,"[-0.5, 148.02, 97.41, 111.23]",eecfe623f6eb45beb9bb08,"적색, 분홍색, 주황색",290101_4120110037623.jpg,416,416,290101,1,"적색, 분홍색, 주황색",391
308128,173056,False,4120140004930,"[-13.63, -7.17, 433.55, 435.23]",81a50f27efd24e4fabdcd5,"적색, 분홍색, 주황색",290101_4120140004930.jpg,416,416,290101,1,"적색, 분홍색, 주황색",391
308129,173056,False,4120150046788,"[-18.1, -10, 453.71, 373.46]",f65423411ae5463c939b85,"적색, 분홍색, 주황색",290101_4120150046788.jpg,416,416,290101,1,"적색, 분홍색, 주황색",391


In [ ]:
train['bbox'] = train['bbox'].apply(ast.literal_eval)
valid['bbox'] = valid['bbox'].apply(ast.literal_eval)

In [ ]:
filtered_train = train[
    ~(train['bbox'].apply(lambda x: x[0] > 416 and x[2] > 0) |
      train['bbox'].apply(lambda x: x[0] > 416 and x[3] > 0) |
      train['bbox'].apply(lambda x: x[1] > 416 and x[2] > 0) |
      train['bbox'].apply(lambda x: x[1] > 416 and x[3] > 0) |
      train['bbox'].apply(lambda x: x[0] < 0 and x[2] < 0) |
      train['bbox'].apply(lambda x: x[0] < 0 and x[3] < 0) |
      train['bbox'].apply(lambda x: x[1] < 0 and x[2] < 0) |
      train['bbox'].apply(lambda x: x[1] < 0 and x[3] < 0))
]
filtered_valid = valid[
    ~(valid['bbox'].apply(lambda x: x[0] > 416 and x[2] > 0) |
      valid['bbox'].apply(lambda x: x[0] > 416 and x[3] > 0) |
      valid['bbox'].apply(lambda x: x[1] > 416 and x[2] > 0) |
      valid['bbox'].apply(lambda x: x[1] > 416 and x[3] > 0) |
      valid['bbox'].apply(lambda x: x[0] < 0 and x[2] < 0) |
      valid['bbox'].apply(lambda x: x[0] < 0 and x[3] < 0) |
      valid['bbox'].apply(lambda x: x[1] < 0 and x[2] < 0) |
      valid['bbox'].apply(lambda x: x[1] < 0 and x[3] < 0))
]

In [5]:
# 데이터셋 폴더 만들기
if os.path.exists("./data/yolo"):
    shutil.rmtree("./data/yolo")

if not os.path.exists("./data/yolo/train"):
    os.makedirs("./data/yolo/train")
    
if not os.path.exists("'./data/yolo/valid"):
    os.makedirs("./data/yolo/valid")

In [5]:
def make_yolo_dataset(df, type="train"):

        for index, row in tqdm(df.iterrows(), total=len(df)):
            image_num = row['image_id']
            code_index = row['Code_Index']
            vienna_code = row['Vienna_Code']
            file_name = row['fileName']
            image_path = os.path.join(f'D:/상표 이미지 및 텍스트/[원천]상표이미지_{type}/{vienna_code}_TOTAL/{file_name}')
            source_image = cv2.imread(image_path, cv2.IMREAD_COLOR) 
            image_height, image_width, _ = source_image.shape
            target_image_path = f"D:/상표 이미지 및 텍스트/Training/data/yolo/{type}/{file_name}"
            cv2.imwrite(target_image_path, source_image)
                           
            target_label_txt = f"D:/상표 이미지 및 텍스트/Training/data/yolo/{type}/{vienna_code}_{image_num}.txt"      
            with open(target_label_txt, "a") as writer:
                x_min, y_min, width, height = row['bbox']
                class_index = code_index
                x_min, y_min = max(0, min(image_width, float(x_min))), max(0, min(image_height, float(y_min)))
                x_max, y_max = max(0, min(image_width, float(x_min + width))), max(0, min(image_height, float(y_min + height)))
                x, y = max(0, min(1, float(((x_min + x_max) / 2) / image_width))), max(0, min(1,float(((y_min + y_max) / 2) / image_height)))
                w, h = max(0, min(1, abs(x_min - x_max) / image_width)), max(0, min(1, abs(y_min - y_max) / image_height))
                writer.write(f"{class_index} {x} {y} {w} {h}\n")

In [ ]:
# 데이터셋 만들기
make_yolo_dataset(filtered_train, "train")
make_yolo_dataset(filtered_valid, "valid")

In [5]:
# 데이터 정보를 담은 yaml 생성하기(클래스 정보 포함)
classes_temp = train['Vienna_Code'].drop_duplicates().tolist() 
classes = {index: class_name for index, class_name in enumerate(classes_temp)}

yaml_data = {
              "names": classes,
              "nc": len(classes),
              "path": "./data/yolo/",
              "train": "train",
              "val": "valid",
            }
with open("./data/yolocustom.yaml", "w") as writer:
    yaml.dump(yaml_data, writer)